In [ ]:
import numpy as np
import pickle
# !pip install bcolz
import bcolz

In [ ]:
from collections import Counter
from functools import reduce
# from tqdm.notebook import tqdm
import string
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
import torch
import urllib
import glob
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import nltk
from PIL import Image
from torchvision import transforms
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet

In [ ]:
from PIL import Image
import pandas as pd
import spacy
from torch.nn.utils.rnn import pad_sequence
spacy_eng = spacy.load("en_core_web_sm")

In [ ]:
def read_img(file):
    img_id=[]
    for img in glob.glob(file+'*.jpg'):
        temp=os.path.basename(img)
        img_id.append(temp)
        
    return img_id

In [ ]:
# extract the data from tar files

In [1]:
# use glove
glove_path = '/scratch/varunc/glove/'
glove_zip = glove_path+'glove.6B.zip'

In [ ]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


# Need not run this again and again


words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=glove_path+'6B.300.dat', mode='w')

with open(glove_path+'glove.6B.300d.txt', 'rb') as f:
    pbar = tqdm(total=file_len(glove_path+'glove.6B.300d.txt'))
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
        pbar.update(1)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 300)), rootdir=glove_path+'6B.300.dat', mode='w')
vectors.flush()
pickle.dump(words, open(glove_path+'6B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(glove_path+'6B.300_idx.pkl', 'wb'))

In [ ]:
vectors = bcolz.open(glove_path+'6B.300.dat')[:]
words = pickle.load(open(glove_path+'6B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(glove_path+'6B.300_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [ ]:
data_imgid=read_img(flickr_path+"Flickr-8K/Flicker8k_Dataset/")


In [ ]:
class Vocabulary:
    def __init__(self, freq_thresh):
        self.freq_thresh = freq_thresh
        self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
        self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
    
    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocab(self, sentence_list):
        lemmatizer = WordNetLemmatizer()
        frequencies = {}
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenizer(sentence):
                lw = lemmatizer.lemmatize(word)
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1
                if frequencies[word] == self.freq_thresh:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1
                    
                if lw not in frequencies:
                    frequencies[lw] = 1
                else:
                    frequencies[lw] += 1
                if frequencies[lw] == self.freq_thresh:
                    self.stoi[lw] = idx
                    self.itos[idx] = lw
                    idx += 1
    
    def numericalize(self, text):
        tok_text = self.tokenizer(text)
        vec = [self.stoi[word] if word in self.stoi else self.stoi["<UNK>"] for word in tok_text]
        return vec

In [ ]:
# for test and train we need to create new text files of the format (Flickr8k.token.txt)
class FlickrDataset(Dataset):
    def __init__(self, root_dir, caption_file, lemma_file, img_file, freq_thresh, transform=None,vocab = None, lemma_vocab=None):
        self.freq_thresh = freq_thresh
        self.transform = transform
        self.root_dir = root_dir
        self.img_file = img_file
        self.caption_file = caption_file
        self.lemma_file = lemma_file
        
        self.caption_dict = self.imgId_caption_dict()
        self.lemma_dict = self.imgId_lemma_dict()
        
        self.imgs , self.captions, self.all_captions = self.load_img_caption()
        if vocab is None:
            self.vocab = Vocabulary(self.freq_thresh)
            self.vocab.build_vocab(self.captions)
        else:
            self.vocab = vocab
            
        self.imgs_lemma , self.lemmas = self.load_img_lemma()
        if lemma_vocab is None:
            self.lemma_vocab = Vocabulary(self.freq_thresh)
            self.lemma_vocab.build_vocab(self.lemmas)
        else:
            self.lemma_vocab = lemma_vocab

    def imgId_caption_dict(self):
        caption_dict = {}
        with open(self.caption_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                temp = line.split()
                img_name, _ = temp[0].split('#')  # first word will be img_id
                description = " ".join([ word for word in temp[1:]]) # get back the description
                if img_name not in caption_dict:
                    caption_dict[img_name] = [description]
                else:
                    caption_dict[img_name].append(description)
        return caption_dict

    def imgId_lemma_dict(self):
        stop_word=list(set(stopwords.words('english')))
        lemma_dict = {}
        with open(self.lemma_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                temp = line.split()
                img_name, _ = temp[0].split('#')  # first word will be img_id
                description = " ".join([ word.lower() for word in temp[1:] if word.lower() not in stop_word]) # get back the description
                description = description.translate(str.maketrans('', '', string.punctuation)).strip()
                if img_name not in lemma_dict:
                    lemma_dict[img_name] = [description]
                else:
                    lemma_dict[img_name].append(description)
        return lemma_dict

    def load_img_caption(self):
        imgs = []
        captions = []
        all_captions = []
        with open(self.img_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                for caption in self.caption_dict[line]:
                    imgs.append(line)
                    captions.append(caption)
                    all_captions.append(self.caption_dict[line])
        return imgs, captions, all_captions

    def load_img_lemma(self):
        imgs = []
        captions = []
        with open(self.img_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                for caption in self.lemma_dict[line]:
                    imgs.append(line)
                    captions.append(caption)
        return imgs, captions

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, index):
        caption = self.captions[index]
        all_caption = self.all_captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption += [self.vocab.stoi["<EOS>"]]
        
        all_num_captions = []
        for caption_i in all_caption:
            num_cap = [self.vocab.stoi["<SOS>"]]
            num_cap += self.vocab.numericalize(caption_i)
            num_cap += [self.vocab.stoi["<EOS>"]]
            all_num_captions += num_cap

        return img, torch.tensor(numericalized_caption), torch.tensor(all_num_captions)

In [3]:
# we can also define simply a function instead of a class
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
    
    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0) # batch * imgs_size * 3 (RGB)
        captions = [item[1] for item in batch] 
        captions = pad_sequence(captions, batch_first=True, padding_value=self.pad_idx)
        all_captions = [item[2] for item in batch]
        all_captions = pad_sequence(all_captions, batch_first=True, padding_value=self.pad_idx)        
        return imgs, captions, all_captions # return the batched images and captions

In [4]:
def get_loader(root_dir, caption_file, lemma_file, img_file, transform, vocab=None, lemma_vocab=None, batch=32, num_worker=2, shuffle=True, pin_memory=True):
    dataset = FlickrDataset(root_dir=root_dir, caption_file=caption_file, lemma_file=lemma_file, img_file=img_file, freq_thresh=2, transform=transform,vocab=vocab, lemma_vocab=lemma_vocab)
    pad_idx = dataset.vocab.stoi["<PAD>"]
    loader = DataLoader(dataset=dataset,
                        batch_size=batch,
                        shuffle=shuffle,
                        collate_fn=MyCollate(pad_idx),
                        pin_memory=pin_memory,
                        num_workers=num_worker)
    return dataset,loader

In [ ]:
trans = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.ToTensor(),
])
dataset, dataloader = get_loader(root_dir=flickr_path+"Flickr-8K/Flicker8k_Dataset",
                        img_file=flickr_path+"Flickr-8K/Flickr_8k.trainImages.txt",
                        caption_file=flickr_path+"Flickr-8K/Flickr8k.token.txt",
                        lemma_file=flickr_path+"Flickr-8K/Flickr8k.lemma.token.txt",
                        transform=trans, batch=512,num_worker=30,
                        shuffle=True)

In [ ]:
test_dataset, test_dataloader = get_loader(root_dir=flickr_path+"Flickr-8K/Flicker8k_Dataset",
                        img_file=flickr_path+"Flickr-8K/Flickr_8k.testImages.txt",
                        caption_file=flickr_path+"Flickr-8K/Flickr8k.token.txt",
                        lemma_file=flickr_path+"Flickr-8K/Flickr8k.lemma.token.txt",
                        transform=trans,
                        vocab=dataset.vocab,
                        lemma_vocab=dataset.lemma_vocab,
                        batch=1, shuffle=True)

## GoogleNet

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        googlenet = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)
        for param in googlenet.parameters():
            param.requires_grad_(False)

        modules = list(googlenet.children())[:-2]
        self.googlenet = nn.Sequential(*modules)
    def forward(self, X):
        x = self.googlenet(X)
        x = x.view(x.shape[0],-1)
        return x

In [ ]:
model = nn.DataParallel(EncoderCNN())
model.eval()
model.to(device)

In [ ]:
def out_vect(filename):
    input_image = Image.open(filename)
    input_tensor = trans(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    input_batch = input_batch.to(device)

    with torch.no_grad():
        output = model(input_batch)
        
    return output

In [ ]:
def out_vect_tensor(input_tensor):
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    input_batch = input_batch.to(device)

    with torch.no_grad():
        output = model(input_batch)
        
    return output

In [ ]:
def get_vector(file,directory):
    img_vector=[]
    imgs_list = []
    with open(file,'r') as f:
        for line in f.readlines():
            x=out_vect(os.path.join(directory,line.strip()))
            img_vector.append(x)
            imgs_list.append(line.strip())
    return imgs_vector, imgs_list

In [ ]:
imgs_vector, imgs_list=get_vector(flickr_path+'Flickr-8K/Flickr_8k.trainImages.txt',
                                  flickr_path+"Flickr-8K/Flicker8k_Dataset/")
imgs_list = np.array(imgs_list)
imgs_tensor = torch.cat(imgs_vector,0)

In [ ]:
def knn(data, test, num=5):
    dist = torch.norm(data - test, dim=1, p=2)
    knn = dist.topk(num, largest=False)
    
    return knn.indices.cpu().numpy()

In [ ]:
def get_words_ll(img_list):
    return [ dataset.lemma_dict[x] for x in img_list]

In [ ]:
def get_attr_encoded(vect, nums=10, train=True):
    if train:
        words_pref_ll = get_words_ll(imgs_list[knn(attr_tensor,vect,nums+1)[1:]])
    else:
        words_pref_ll = get_words_ll(imgs_list[knn(attr_tensor,vect,nums)])
    all_words = ' '.join(reduce(lambda x,y: x+y,words_pref_ll)).split()
    all_words = [ x for x in all_words if x in dataset.vocab.stoi]
    top_words = [ x for x,y in Counter(all_words).most_common(nums)]
    return top_words

In [ ]:
def get_attr_tensor(img_tensor,train=True):
    model.to(device)
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        vect = model(img_tensor)
    return get_attr_encoded(vect, train)

## Embedding Layer using GLoVE

In [ ]:
target_vocab = dataset.vocab.itos

In [ ]:
matrix_len = len(target_vocab)  # target vocabulary 
weights_matrix = np.zeros((matrix_len, 300))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))
weights_matrix = torch.tensor(weights_matrix,requires_grad = False).to(device)

In [ ]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim).to(device)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings

## Code for Decoder RNN

In [ ]:
class DecoderRNN(nn.Module):
    """
    takes as input img vector and captions
    """
    def __init__(self):
        super().__init__()
        self.input_size = 512  # m = input_size in paper
        self.hidden_size = 512 # n = hidden_size in paper
        self.word_size = 300
        self.image_size = 1024
        self.lstm = nn.LSTMCell(self.input_size, self.hidden_size) # using single cell rather than lstm
        self.U = nn.Linear(self.word_size, self.word_size)
        self.V = nn.Linear(self.hidden_size, self.word_size)
        
        self.W_x_Y = nn.Linear(self.word_size, self.input_size)
        self.W_Y_h = nn.Linear(self.hidden_size, self.word_size)
        self.image_mapping = nn.Linear(self.image_size, self.input_size)
        
        self.W_x_A = nn.Linear(self.word_size, self.word_size)
        self.W_Y_A = nn.Linear(self.word_size, self.word_size)
        
        # use the pretrained glove embedding
        self.vocab, self.num_embedding = create_emb_layer(weights_matrix, True)  # num_embeddings x 300
        self.soft = nn.Softmax(dim=1)
        self.tanh = nn.Tanh()
        self.ReLU = nn.ReLU(inplace = True)
        self.vocab = self.vocab.to(device)

    def input_attention(self, previous_word, attributes):
        """ 
        Apply input attention at each step.
        attributes -> matrix where columns store the word embedding, batch x no_of_attr x 300
        previous word -> batch x 300
        """
        # for each word in the vocabulary find the score
        score = torch.bmm(self.U(previous_word).unsqueeze(1), attributes.transpose(1,2))
        score = score.squeeze(1) # batch x no_of_attributes
        score = self.soft(score)  # batch x no_of_attributes
        weighted_y = torch.bmm(score.unsqueeze(1), attributes).squeeze(1)  # batch x 300
        
        scaled_y = self.W_x_A(weighted_y)
        final_y = scaled_y + previous_word # batch x 300
        x = self.W_x_Y(final_y)  # batch x 512
        return x, score
        
    def output_attention(self, hidden_state, attributes):
        """
        Apply output attention at each step
        hidden_state = current hidden state of LSTMCell, used for predicting current output (batch x 512)
        attributes -> matrix where columns store the word embedding, 300 x no_of_attr
        """
        score = torch.bmm(self.V(hidden_state).unsqueeze(1), self.tanh(attributes).transpose(1,2)) # batch x 1 x no_of_attributes
        score = self.soft(score)  # batch x no_of_attributes
        weighted_y = torch.bmm(score, self.tanh(attributes)) # batch x 1 x 300
        weighted_y = weighted_y.squeeze(1) # batch x 300
        
        y_to_h = self.W_Y_A(weighted_y)
        hidden = self.W_Y_h(hidden_state)  # batch x 300
        hidden = hidden + y_to_h  # batch x 300
        logits = torch.mm(hidden, self.vocab.weight.transpose(0,1)) # batch x vocab_size
        score = score.squeeze(1) # batch x no_of_attr
        return logits, score

    def prepare_attributes(self, attr):
        no_of_attr = attr.shape[1] # batch x no_of_attr
        batch = attr.shape[0] 
        attributes = torch.zeros(batch, no_of_attr, 300).to(device)  # batch x no_of_attr x 300
        for i in range(batch):
            for j in range(no_of_attr):
                attributes[i, j, :] = self.vocab(attr[i, j])
        return attributes

    def next_word(self, logits):
        """
        Samples from the top 3 words randomly and returns a word
        """
        probability = torch.softmax(logits, dim=1)  # batch x no_of_attr
        prob_sort, indices = torch.sort(probability, descending=True)
        # generate a array of size -> batch
        arr = torch.randint(3, (logits.shape[0],)).to(device)
        random_words = torch.zeros(logits.shape[0], 300).to(device)
        random_idxs = torch.zeros(logits.shape[0]).to(device)

        for i in range(logits.shape[0]):
            ind = indices[i, arr[i]]
            random_words[i] = self.vocab(ind)
            random_idxs[i] =ind
        return random_words, random_idxs  # batch x 300

    def forward(self, seq_len, image_vectors, attributes, caption_ids=None, train=True):
        """
        image_vectors -> batch x img_size
        caption_ids -> batch x seq_len x word_size (make it not None, if you want to do teacher forcing)
        attributes -> batch x 5, we have 5 attributes for each of the image (contains indices of words in our dictionary)
        it should also contain <PAD>, <SOS>, <EOS> as well. 
        """
        image_vectors = self.ReLU(self.image_mapping(image_vectors)) # batch x 512
        attributes = self.prepare_attributes(attributes)
        batch_size = image_vectors.shape[0]

        Hidden_logits = torch.zeros(batch_size, seq_len, self.vocab.weight.shape[0]).to(device)  # batch x seq x vocab_size
        Hidden_scores = torch.zeros(batch_size, seq_len, attributes.shape[1]).to(device) # batch x seq x no_of_attr
        Input_scores = torch.zeros(batch_size, seq_len, attributes.shape[1]).to(device) # batch x seq x no_of_attr

        words = []
#         word = self.initial_attention(image_vectors)
        
        hidden, cell = self.lstm(image_vectors) # batch x 512
        logits, hidden_score = self.output_attention(hidden, attributes)
        word, word_ind = self.next_word(logits)  # batch x 300
#         words.append(word_ind)

        for i in range(seq_len):
            # prepare the input using input attention
            next_input, input_score = self.input_attention(word, attributes)
            Input_scores[:, i, :] = input_score
            hidden, cell = self.lstm(next_input, (hidden, cell))
            logits, hidden_score = self.output_attention(hidden, attributes)
            Hidden_scores[:, i, :] = hidden_score
            Hidden_logits[:, i, :] = logits
            word, word_ind = self.next_word(logits)  # batch x 300
            words.append(word_ind)
            if not train and word_ind == dataset.vocab.stoi['<EOS>']:
                break

        return Hidden_logits, Hidden_scores, Input_scores, words

In [ ]:
# loss function
class MyLoss(nn.Module):
    """Crossentropy + regularization"""
    def __init__(self):
        super().__init__()
        self.criterion = nn.CrossEntropyLoss(reduction='none').to(device)

    def forward(self, output, target, Hidden_scores, Input_scores):
        # output -> batch x seq x no_of_attr
        # target -> batch x seq
        x = self.criterion(output, target).mean(dim=1)  # batch

        hidden_reg = self.regularize(Hidden_scores)
        input_reg = self.regularize(Input_scores)

        ret = x+1e-5*(hidden_reg+input_reg)
        ret = ret.mean()
        return ret  # equation 10 in paper

    def regularize(self, score):
        # batch x time x i(attr index)
        s1 = torch.sum(score, dim=1)
        s1 = s1 * s1
        s1 = torch.sqrt(torch.sum(s1, dim=1)) # sum_i_()^{0.5}

        s2 = torch.sqrt(score)
        s2 = torch.sum(s2, dim=2)
        s2 = s2 * s2
        s2 = torch.sum(s2, dim=1)  # sum_time_()^{2}
        return s1 + s2  # equation 11 in paper

In [ ]:
encoder = nn.DataParallel(EncoderCNN())
decoder = nn.DataParallel(DecoderRNN())
encoder.to(device)
decoder.to(device)

In [ ]:
try:
    encoder.module.load_state_dict(torch.load('encoder dict.pth'))
except:
    encoder.load_state_dict(torch.load('encoder dict.pth'))
try:
    decoder.module.load_state_dict(torch.load('decoder dict.pth'))
except:
    decoder.load_state_dict(torch.load('decoder dict.pth'))

In [ ]:
NUM_EPOCHS = 500
criterion = MyLoss().to(device)
optimizer = torch.optim.Adam(params = decoder.parameters(), lr = 0.0001, weight_decay=1e-6)

## Training Loop

In [ ]:
encoder.train()
decoder.train()
for epoch in range(NUM_EPOCHS):
    pbar = tqdm(total = len(dataloader),desc='Loss: -')
    net_loss = 0
    for idx, (img, caption, all_caption) in enumerate(dataloader):
        img = img.to(device)
        caption = caption.to(device) # seq_len x batch_size
        
        decoder.zero_grad()
        encoder.zero_grad()
        
        encode_out = encoder(img)
        try:
            x = []
            for out_idx in range(encode_out.shape[0]):
                x.append([ test_dataset.vocab.stoi[a] for a in get_attr_encoded(encode_out[out_idx][None,:])])

            attrs = torch.tensor(x).to(device)
        except:
            x = []
            for out_idx in range(encode_out.shape[0]):
                x.append(' '.join(get_attr_encoded(encode_out[out_idx][None,:])))

            print(x)
            print(np.vectorize(lambda x: dataset.vocab.itos[x])(capts.cpu().numpy().squeeze()))
            raise ValueError
            
        hidden_logits, hidden_scores, input_scores, _ = decoder(caption.shape[1], encode_out, attrs)
        loss = criterion(hidden_logits.transpose(1,2), caption, hidden_scores, input_scores)
        loss.backward()
        
        optimizer.step()
        
        net_loss += loss.cpu().item()
        
        pbar.update(1)
        pbar.set_description(f'Epoch: {epoch+1} Loss {net_loss*1.0/(idx+1):.2f}')
        
    pbar.refresh()
    if (epoch+1)%20==0:
        torch.save(encoder.state_dict(),f'encoder dict {epoch+1}.pth')
        torch.save(decoder.state_dict(),f'decoder dict {epoch+1}.pth')